# ES UAT BAN Prep for Migration Testing

This notebook prepares ES UAT BANs for migration testing by marking them with `BBF_Ban__c = True`.

## Purpose
In production, the `es_to_bbf_ban_creation_v12.ipynb` creates new -BBF BANs in OSS and marks them for migration.
Since UAT doesn't have OSS access, this notebook mimics that process by:
1. Identifying qualifying Orders (Active + NOT PA MARKET DECOM)
2. Finding their parent BANs
3. Marking those BANs with `BBF_Ban__c = True`
4. Extracting related Account and Address IDs for downstream migrations

## Central Policy (Same as Production)
**Include Orders where:**
- Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')

**Exclude Orders where:**
- Project_Group__c LIKE '%PA MARKET DECOM%'

All other Project_Group__c values (including NULL or any other project) are **included**.

## Output
- Updates ES UAT `Billing_Invoice__c.BBF_Ban__c = True` for qualifying BANs
- Excel file with:
  - BANs marked for migration
  - Account IDs to migrate
  - Address IDs to migrate
  - Order counts and summary

## Safety
- `DRY_RUN = True` by default (shows what would be updated)
- `BAN_LIMIT` controls how many BANs to mark (default: 20 for testing)
- `MAX_ORDERS_PER_BAN` limits BANs to those with few orders (default: 5 for POC)
- Only marks BANs, doesn't create or delete anything

In [ ]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
from collections import defaultdict
import dotenv
import os

# Load environment variables from .env file
dotenv.load_dotenv()

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("Setup complete")

Python: C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Pandas: 2.2.3
Setup complete


In [ ]:
# === CONFIGURATION ===

# ES UAT Credentials
ES_USERNAME = "sfdcapi@everstream.net.uat"
ES_PASSWORD = "AS12df34!@#$"
ES_TOKEN = "X0ation2CNmK5C0pV94M6vFYS"
ES_DOMAIN = "test"

# === MIGRATION POLICY (same as production) ===
ACTIVE_STATUSES = ["Activated", "Suspended (Late Payment)", "Disconnect in Progress"]
WD_PROJECT_GROUP = "PA MARKET DECOM"  # Orders with this are EXCLUDED

# === RUN OPTIONS ===
DRY_RUN = False  # Set to False to actually update BANs
BAN_LIMIT = 20  # Max BANs to mark (0 = no limit)
MAX_ORDERS_PER_BAN = (
    5  # Max orders per BAN (0 = no limit) - keeps POC small and easy to undo
)

# === OUTPUT ===
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_type = "dry_run" if DRY_RUN else "live_run"
output_file = f"uat_ban_prep_{run_type}_{timestamp}.xlsx"

print("=" * 70)
print("CONFIGURATION")
print("=" * 70)
print(f"DRY_RUN: {DRY_RUN}")
print(f"BAN_LIMIT: {BAN_LIMIT if BAN_LIMIT > 0 else 'No limit'}")
print(
    f"MAX_ORDERS_PER_BAN: {MAX_ORDERS_PER_BAN if MAX_ORDERS_PER_BAN > 0 else 'No limit'}"
)
print(f"Output: {output_file}")
print(f"\nMigration Policy:")
print(f"  Active Statuses: {ACTIVE_STATUSES}")
print(f"  Exclude: {WD_PROJECT_GROUP}")

if not DRY_RUN:
    print("\n" + "!" * 70)
    print("WARNING: LIVE RUN - BANs will be updated with BBF_Ban__c = True")
    print("!" * 70)

In [ ]:
# === CONNECT TO ES UAT ===

print("=" * 70)
print("CONNECTING TO ES UAT")
print("=" * 70)

print("\nConnecting to ES UAT Salesforce...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"Connected to ES UAT: {es_sf.sf_instance}")

CONNECTING TO ES UAT

Connecting to ES UAT Salesforce...
Connected to ES UAT: everstream--uat.sandbox.my.salesforce.com


In [31]:
# === STEP 1: QUERY QUALIFYING ORDERS ===
# Orders that meet the migration criteria:
# - Status IN (Active statuses)
# - EXCLUDE: Project_Group__c LIKE '%PA MARKET DECOM%'
# All other Project_Group__c values (including NULL) are included.

print("\n" + "=" * 70)
print("STEP 1: QUERY QUALIFYING ORDERS")
print("=" * 70)

status_str = "','".join(ACTIVE_STATUSES)

# Note: The (Project_Group__c = null OR NOT LIKE) pattern is required because
# SOQL NULL comparisons don't work with NOT LIKE alone
orders_query = f"""
    SELECT Id, OrderNumber, Name, Service_ID__c,
           Status, Project_Group__c,
           Billing_Invoice__c, Billing_Invoice__r.Name, Billing_Invoice__r.Account__c,
           Billing_Invoice__r.Account__r.Name, Billing_Invoice__r.BBF_Ban__c,
           AccountId, Account.Name,
           Address_A__c, Address_A__r.Name,
           Address_Z__c, Address_Z__r.Name
    FROM Order
    WHERE Status IN ('{status_str}')
      AND (Project_Group__c = null OR (NOT Project_Group__c LIKE '%{WD_PROJECT_GROUP}%'))
      AND Billing_Invoice__c != null
    ORDER BY Billing_Invoice__r.Name
"""

print(f"Criteria:")
print(f"  Include: Status IN {ACTIVE_STATUSES}")
print(f"  Exclude: Project_Group__c LIKE '%{WD_PROJECT_GROUP}%'")
print(f"\nExecuting query...")

result = es_sf.query_all(orders_query)
qualifying_orders = result["records"]

print(f"\nFound {len(qualifying_orders):,} qualifying Orders")

if len(qualifying_orders) > 0:
    sample = qualifying_orders[0]
    print(f"\nSample Order:")
    print(f"  Order Number: {sample.get('OrderNumber', 'N/A')}")
    print(f"  Service ID:   {sample.get('Service_ID__c', 'N/A')}")
    print(f"  Status:       {sample.get('Status', 'N/A')}")
    print(f"  Project Group: {sample.get('Project_Group__c', '(none)')}")
    print(f"  BAN:          {sample.get('Billing_Invoice__r', {}).get('Name', 'N/A')}")
    print(f"  Account:      {sample.get('Account', {}).get('Name', 'N/A')}")


STEP 1: QUERY QUALIFYING ORDERS
Criteria:
  Include: Status IN ['Activated', 'Suspended (Late Payment)', 'Disconnect in Progress']
  Exclude: Project_Group__c LIKE '%PA MARKET DECOM%'

Executing query...

Found 13,535 qualifying Orders

Sample Order:
  Order Number: SOF-78413
  Service ID:   EV-DFBR-78413
  Status:       Activated
  Project Group: None
  BAN:          00-BUCK
  Account:      Buckeye Telesystem


In [32]:
# === STEP 2: EXTRACT UNIQUE IDs ===
# Get unique BANs, Accounts, and Addresses from qualifying Orders

print("\n" + "=" * 70)
print("STEP 2: EXTRACT UNIQUE IDs")
print("=" * 70)

# Collect unique IDs
ban_data = (
    {}
)  # BAN SFID -> {name, account_id, account_name, order_count, already_marked}
account_ids = set()
address_ids = set()

for order in qualifying_orders:
    # BAN
    ban_id = order.get("Billing_Invoice__c")
    if ban_id:
        if ban_id not in ban_data:
            ban_info = order.get("Billing_Invoice__r", {}) or {}
            account_info = ban_info.get("Account__r", {}) or {}
            ban_data[ban_id] = {
                "ban_id": ban_id,
                "ban_name": ban_info.get("Name", "N/A"),
                "account_id": ban_info.get("Account__c"),
                "account_name": account_info.get("Name", "N/A"),
                "already_marked": ban_info.get("BBF_Ban__c", False),
                "order_count": 0,
                "orders": [],
            }
        ban_data[ban_id]["order_count"] += 1
        ban_data[ban_id]["orders"].append(order.get("OrderNumber", "N/A"))

    # Account (from BAN's Account, not Order's Account)
    ban_info = order.get("Billing_Invoice__r", {}) or {}
    if ban_info.get("Account__c"):
        account_ids.add(ban_info["Account__c"])

    # Addresses
    if order.get("Address_A__c"):
        address_ids.add(order["Address_A__c"])
    if order.get("Address_Z__c"):
        address_ids.add(order["Address_Z__c"])

# Convert to list and sort by order count
ban_list = sorted(ban_data.values(), key=lambda x: x["order_count"], reverse=True)

print(f"\nUnique counts from qualifying Orders:")
print(f"  BANs:      {len(ban_list):,}")
print(f"  Accounts:  {len(account_ids):,}")
print(f"  Addresses: {len(address_ids):,}")

# Check how many are already marked
already_marked = [b for b in ban_list if b["already_marked"]]
not_marked = [b for b in ban_list if not b["already_marked"]]

print(f"\nBAN Status:")
print(f"  Already BBF_Ban__c = True: {len(already_marked):,}")
print(f"  Not yet marked:            {len(not_marked):,}")

print(f"\nTop 10 BANs by order count:")
for i, ban in enumerate(ban_list[:10], 1):
    marked = "[MARKED]" if ban["already_marked"] else ""
    print(
        f"  {i}. {ban['ban_name']} - {ban['order_count']} orders - {ban['account_name']} {marked}"
    )


STEP 2: EXTRACT UNIQUE IDs

Unique counts from qualifying Orders:
  BANs:      2,512
  Accounts:  2,265
  Addresses: 11,331

BAN Status:
  Already BBF_Ban__c = True: 0
  Not yet marked:            2,512

Top 10 BANs by order count:
  1. E91910226264-W - 1170 orders - DISH Wireless 
  2. E91910165108-R - 744 orders - T-Mobile 
  3. E91910205540-R - 551 orders - T-Mobile Small Cell 
  4. E91910159507-W - 480 orders - T-Mobile 
  5. A91910221190-R - 326 orders - Meijer, Inc 
  6. E91910167302-R - 306 orders - T-Mobile 
  7. A91910155661-W - 273 orders - CenturyLink - dba Lumen 
  8. A91910235620-W - 272 orders - Verizon Wireless - Customer 
  9. A91910197622-W - 268 orders - AT&T Mobility 
  10. E91910156370-R - 245 orders - Northeast Ohio Network for Educational Technology-NEONET 


In [33]:
# === STEP 3: SELECT BANs TO MARK ===
# Apply filters:
# 1. Filter out already-marked BANs
# 2. Filter by MAX_ORDERS_PER_BAN (keeps POC manageable)
# 3. Apply BAN_LIMIT

print("\n" + "=" * 70)
print("STEP 3: SELECT BANs TO MARK")
print("=" * 70)

# Only mark BANs that aren't already marked
bans_to_mark = not_marked.copy()

# Filter by max orders per BAN (for POC testing)
if MAX_ORDERS_PER_BAN > 0:
    before_filter = len(bans_to_mark)
    bans_to_mark = [b for b in bans_to_mark if b["order_count"] <= MAX_ORDERS_PER_BAN]
    print(f"\nFiltering by MAX_ORDERS_PER_BAN <= {MAX_ORDERS_PER_BAN}:")
    print(f"  Before: {before_filter:,} BANs")
    print(f"  After:  {len(bans_to_mark):,} BANs")

# Sort by order count ascending (smallest first for POC)
bans_to_mark = sorted(bans_to_mark, key=lambda x: x["order_count"])

# Apply BAN limit
if BAN_LIMIT > 0 and len(bans_to_mark) > BAN_LIMIT:
    print(f"\nApplying BAN_LIMIT: {BAN_LIMIT} BANs")
    bans_to_mark = bans_to_mark[:BAN_LIMIT]

print(f"\nBANs selected for marking: {len(bans_to_mark)}")

# Recalculate related IDs based on selected BANs
selected_ban_ids = {b["ban_id"] for b in bans_to_mark}
selected_account_ids = {b["account_id"] for b in bans_to_mark if b["account_id"]}

# Get addresses only for orders on selected BANs
selected_address_ids = set()
selected_order_count = 0
for order in qualifying_orders:
    if order.get("Billing_Invoice__c") in selected_ban_ids:
        selected_order_count += 1
        if order.get("Address_A__c"):
            selected_address_ids.add(order["Address_A__c"])
        if order.get("Address_Z__c"):
            selected_address_ids.add(order["Address_Z__c"])

print(f"\nSelected migration scope:")
print(f"  BANs:      {len(bans_to_mark):,}")
print(f"  Accounts:  {len(selected_account_ids):,}")
print(f"  Addresses: {len(selected_address_ids):,}")
print(f"  Orders:    {selected_order_count:,}")

if len(bans_to_mark) > 0:
    print(f"\nBANs to mark:")
    for i, ban in enumerate(bans_to_mark[:20], 1):
        print(
            f"  {i}. {ban['ban_name']} ({ban['order_count']} orders) - {ban['account_name']}"
        )
    if len(bans_to_mark) > 20:
        print(f"  ... and {len(bans_to_mark) - 20} more")
else:
    print(
        "\nNo BANs to mark (all qualifying BANs already have BBF_Ban__c = True or exceed MAX_ORDERS_PER_BAN)"
    )


STEP 3: SELECT BANs TO MARK

Filtering by MAX_ORDERS_PER_BAN <= 5:
  Before: 2,512 BANs
  After:  2,256 BANs

Applying BAN_LIMIT: 20 BANs

BANs selected for marking: 20

Selected migration scope:
  BANs:      20
  Accounts:  20
  Addresses: 31
  Orders:    20

BANs to mark:
  1. 00-BUCK (1 orders) - Buckeye Telesystem
  2. 00-CCI (1 orders) - CCI Systems
  3. 00-D0420 (1 orders) - D&P Communications
  4. 00-LFC (1 orders) - Lansing Fiber Communications - Zayo
  5. 00-MCISD (1 orders) - Monroe County ISD
  6. 00-MN (1 orders) - Merit Network Inc.
  7. 00-PACKER (1 orders) - CCI Systems Inc. dba Packerland Broadband
  8. A91910101079-R (1 orders) - Muskegon Family Care
  9. A91910101111-R (1 orders) - Otsego Memorial Hospital
  10. A91910101327-R (1 orders) - Bronson Healthcare Group
  11. A91910101368-R (1 orders) - Alcona Health Center
  12. A91910101426-R (1 orders) - Wolverine Mutual Insurance
  13. A91910101608-W (1 orders) - Windstream
  14. A91910101830-R (1 orders) - Mourer Fost

In [34]:
# === STEP 4: UPDATE BANs ===
# Set BBF_Ban__c = True on selected BANs

print("\n" + "=" * 70)
print("STEP 4: UPDATE BANs")
print("=" * 70)

update_results = []

if len(bans_to_mark) == 0:
    print("\nNo BANs to update.")
elif DRY_RUN:
    print(f"\nDRY RUN - Would update {len(bans_to_mark)} BANs")
    for ban in bans_to_mark:
        update_results.append(
            {
                "ban_id": ban["ban_id"],
                "ban_name": ban["ban_name"],
                "account_name": ban["account_name"],
                "order_count": ban["order_count"],
                "status": "WOULD_UPDATE",
                "error": None,
            }
        )
    print("\nNo changes made (DRY_RUN = True)")
else:
    print(f"\nUpdating {len(bans_to_mark)} BANs with BBF_Ban__c = True...")

    # Build update records
    updates = [{"Id": ban["ban_id"], "BBF_Ban__c": True} for ban in bans_to_mark]

    try:
        results = es_sf.bulk.Billing_Invoice__c.update(updates)

        success_count = 0
        error_count = 0

        for i, result in enumerate(results):
            ban = bans_to_mark[i]
            if result["success"]:
                success_count += 1
                update_results.append(
                    {
                        "ban_id": ban["ban_id"],
                        "ban_name": ban["ban_name"],
                        "account_name": ban["account_name"],
                        "order_count": ban["order_count"],
                        "status": "UPDATED",
                        "error": None,
                    }
                )
            else:
                error_count += 1
                update_results.append(
                    {
                        "ban_id": ban["ban_id"],
                        "ban_name": ban["ban_name"],
                        "account_name": ban["account_name"],
                        "order_count": ban["order_count"],
                        "status": "ERROR",
                        "error": str(result["errors"]),
                    }
                )

        print(f"\nUpdate Results:")
        print(f"  Successfully updated: {success_count}")
        print(f"  Errors: {error_count}")

        if error_count > 0:
            print(f"\nFirst 5 errors:")
            for r in update_results:
                if r["status"] == "ERROR":
                    print(f"  - {r['ban_name']}: {r['error']}")

    except Exception as e:
        print(f"\nError during bulk update: {e}")
        for ban in bans_to_mark:
            update_results.append(
                {
                    "ban_id": ban["ban_id"],
                    "ban_name": ban["ban_name"],
                    "account_name": ban["account_name"],
                    "order_count": ban["order_count"],
                    "status": "ERROR",
                    "error": str(e),
                }
            )


STEP 4: UPDATE BANs

Updating 20 BANs with BBF_Ban__c = True...

Update Results:
  Successfully updated: 20
  Errors: 0


In [35]:
# === STEP 5: CREATE EXCEL OUTPUT ===

print("\n" + "=" * 70)
print("STEP 5: CREATE EXCEL OUTPUT")
print("=" * 70)

wb = Workbook()

# Styling
header_font = Font(bold=True, color="FFFFFF")
header_fill = PatternFill("solid", fgColor="4472C4")
header_fill_green = PatternFill("solid", fgColor="70AD47")
header_fill_orange = PatternFill("solid", fgColor="ED7D31")
header_alignment = Alignment(horizontal="center", vertical="center")
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)


def style_header(ws, headers, fill_color):
    for col, header in enumerate(headers, 1):
        cell = ws.cell(row=1, column=col)
        cell.font = header_font
        cell.fill = fill_color
        cell.alignment = header_alignment
        cell.border = thin_border


def auto_width(ws):
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        ws.column_dimensions[column].width = min(max_length + 2, 50)


status_colors = {
    "WOULD_UPDATE": "C6EFCE",
    "UPDATED": "C6EFCE",
    "ERROR": "FF6666",
    "ALREADY_MARKED": "FFEB9C",
}

# === Sheet 1: Summary ===
ws1 = wb.active
ws1.title = "Summary"
ws1.append(["ES UAT BAN Prep Summary"])
ws1["A1"].font = Font(bold=True, size=14)
ws1.append([])
ws1.append(["Run Type:", "DRY RUN" if DRY_RUN else "LIVE RUN"])
ws1.append(["BAN Limit:", BAN_LIMIT if BAN_LIMIT > 0 else "No limit"])
ws1.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws1.append([])
ws1.append(["Migration Policy:"])
ws1["A7"].font = Font(bold=True)
ws1.append(["  Active Statuses:", ", ".join(ACTIVE_STATUSES)])
ws1.append(["  Exclude Project Group:", WD_PROJECT_GROUP])
ws1.append([])
ws1.append(["Qualifying Orders Found:", len(qualifying_orders)])
ws1.append([])
ws1.append(["Unique Counts (All Qualifying):"])
ws1["A13"].font = Font(bold=True)
ws1.append(["  Total BANs:", len(ban_list)])
ws1.append(["  Already Marked:", len(already_marked)])
ws1.append(["  Not Yet Marked:", len(not_marked)])
ws1.append(["  Total Accounts:", len(account_ids)])
ws1.append(["  Total Addresses:", len(address_ids)])
ws1.append([])
ws1.append(["Selected for This Run:"])
ws1["A20"].font = Font(bold=True)
ws1.append(["  BANs to Mark:", len(bans_to_mark)])
ws1.append(["  Related Accounts:", len(selected_account_ids)])
ws1.append(["  Related Addresses:", len(selected_address_ids)])
ws1.append(["  Related Orders:", selected_order_count])
ws1.append([])
if update_results:
    ws1.append(["Update Results:"])
    ws1[f"A{ws1.max_row}"].font = Font(bold=True)
    status_counts = defaultdict(int)
    for r in update_results:
        status_counts[r["status"]] += 1
    for status, count in status_counts.items():
        ws1.append([f"  {status}:", count])

# === Sheet 2: BANs to Migrate ===
ws2 = wb.create_sheet("BANs to Migrate")
headers2 = [
    "BAN ID",
    "BAN Name",
    "Account ID",
    "Account Name",
    "Order Count",
    "Status",
    "Error",
]
ws2.append(headers2)
style_header(ws2, headers2, header_fill)

for r in update_results:
    ban = next((b for b in bans_to_mark if b["ban_id"] == r["ban_id"]), {})
    ws2.append(
        [
            r["ban_id"],
            r["ban_name"],
            ban.get("account_id", ""),
            r["account_name"],
            r["order_count"],
            r["status"],
            r["error"] or "",
        ]
    )
    row_idx = ws2.max_row
    fill_color = status_colors.get(r["status"], "FFFFFF")
    for col in range(1, len(headers2) + 1):
        ws2.cell(row=row_idx, column=col).fill = PatternFill(
            "solid", fgColor=fill_color
        )

ws2.freeze_panes = "A2"
auto_width(ws2)

# === Sheet 3: Already Marked BANs ===
ws3 = wb.create_sheet("Already Marked")
headers3 = ["BAN ID", "BAN Name", "Account ID", "Account Name", "Order Count"]
ws3.append(headers3)
style_header(ws3, headers3, header_fill_orange)

for ban in already_marked:
    ws3.append(
        [
            ban["ban_id"],
            ban["ban_name"],
            ban["account_id"],
            ban["account_name"],
            ban["order_count"],
        ]
    )

ws3.freeze_panes = "A2"
auto_width(ws3)

# === Sheet 4: Account IDs ===
ws4 = wb.create_sheet("Account IDs")
headers4 = ["Account ID"]
ws4.append(headers4)
style_header(ws4, headers4, header_fill_green)

for acc_id in sorted(selected_account_ids):
    ws4.append([acc_id])

ws4.freeze_panes = "A2"
auto_width(ws4)

# === Sheet 5: Address IDs ===
ws5 = wb.create_sheet("Address IDs")
headers5 = ["Address ID"]
ws5.append(headers5)
style_header(ws5, headers5, header_fill_green)

for addr_id in sorted(selected_address_ids):
    ws5.append([addr_id])

ws5.freeze_panes = "A2"
auto_width(ws5)

# === Sheet 6: All Qualifying Orders ===
ws6 = wb.create_sheet("Qualifying Orders")
headers6 = [
    "Order Number",
    "Service ID",
    "Status",
    "BAN Name",
    "Account Name",
    "Address A",
    "Address Z",
]
ws6.append(headers6)
style_header(ws6, headers6, header_fill)

for order in qualifying_orders:
    # Only include orders for selected BANs
    if order.get("Billing_Invoice__c") in selected_ban_ids:
        ws6.append(
            [
                order.get("OrderNumber", ""),
                order.get("Service_ID__c", ""),
                order.get("Status", ""),
                (
                    order.get("Billing_Invoice__r", {}).get("Name", "")
                    if order.get("Billing_Invoice__r")
                    else ""
                ),
                (
                    order.get("Account", {}).get("Name", "")
                    if order.get("Account")
                    else ""
                ),
                order.get("Address_A__c", ""),
                order.get("Address_Z__c", ""),
            ]
        )

ws6.freeze_panes = "A2"
auto_width(ws6)

# Save
wb.save(output_file)
print(f"\nExcel output saved to: {output_file}")
print(f"  Sheet 1: Summary")
print(f"  Sheet 2: BANs to Migrate ({len(update_results)} BANs)")
print(f"  Sheet 3: Already Marked ({len(already_marked)} BANs)")
print(f"  Sheet 4: Account IDs ({len(selected_account_ids)} accounts)")
print(f"  Sheet 5: Address IDs ({len(selected_address_ids)} addresses)")
print(f"  Sheet 6: Qualifying Orders ({selected_order_count} orders)")


STEP 5: CREATE EXCEL OUTPUT

Excel output saved to: uat_ban_prep_live_run_20260114_090759.xlsx
  Sheet 1: Summary
  Sheet 2: BANs to Migrate (20 BANs)
  Sheet 3: Already Marked (0 BANs)
  Sheet 4: Account IDs (20 accounts)
  Sheet 5: Address IDs (31 addresses)
  Sheet 6: Qualifying Orders (20 orders)


In [36]:
# === FINAL SUMMARY ===

print("\n" + "=" * 70)
print("UAT BAN PREP COMPLETE")
print("=" * 70)

print(f"\nRun Type: {'DRY RUN' if DRY_RUN else 'LIVE RUN'}")
print(f"\nMigration Scope (selected BANs):")
print(f"  BANs:      {len(bans_to_mark):,}")
print(f"  Accounts:  {len(selected_account_ids):,}")
print(f"  Addresses: {len(selected_address_ids):,}")
print(f"  Orders:    {selected_order_count:,}")

print(f"\nOutput file: {output_file}")

if DRY_RUN:
    print("\n" + "-" * 70)
    print("DRY RUN complete. No BANs were updated.")
    print("To mark BANs for migration, set DRY_RUN = False and re-run.")
    print("-" * 70)
else:
    print("\n" + "-" * 70)
    print("LIVE RUN complete. BANs have been marked with BBF_Ban__c = True.")
    print("\nNext Steps:")
    print("  1. Run 02_account_migration.ipynb (filter by Account IDs in Excel)")
    print("  2. Run 03_contact_migration.ipynb")
    print("  3. Run 01_location_migration.ipynb (filter by Address IDs in Excel)")
    print("  4. Run 04_ban_migration.ipynb (will pick up BBF_Ban__c = true)")
    print("  5. Run 05_service_migration.ipynb")
    print("  6. Run 06_service_charge_migration.ipynb")
    print("  7. Run 07_offnet_migration.ipynb")
    print("-" * 70)


UAT BAN PREP COMPLETE

Run Type: LIVE RUN

Migration Scope (selected BANs):
  BANs:      20
  Accounts:  20
  Addresses: 31
  Orders:    20

Output file: uat_ban_prep_live_run_20260114_090759.xlsx

----------------------------------------------------------------------
LIVE RUN complete. BANs have been marked with BBF_Ban__c = True.

Next Steps:
  1. Run 02_account_migration.ipynb (filter by Account IDs in Excel)
  2. Run 03_contact_migration.ipynb
  3. Run 01_location_migration.ipynb (filter by Address IDs in Excel)
  4. Run 04_ban_migration.ipynb (will pick up BBF_Ban__c = true)
  5. Run 05_service_migration.ipynb
  6. Run 06_service_charge_migration.ipynb
  7. Run 07_offnet_migration.ipynb
----------------------------------------------------------------------


---
## Next Steps

After running this notebook with `DRY_RUN = False`:

### Option A: Use the Excel Output to Filter Other Notebooks
The Excel file contains:
- **Account IDs sheet**: Use these to filter `02_account_migration.ipynb`
- **Address IDs sheet**: Use these to filter `01_location_migration.ipynb`

### Option B: Modify Other Notebooks (Recommended)
Modify the Account and Location notebooks to query based on BBF_Ban__c:

```python
# Account migration - only migrate Accounts linked to BBF_Ban__c = true BANs
WHERE Id IN (
    SELECT Account__c FROM Billing_Invoice__c WHERE BBF_Ban__c = true
)

# Location migration - only migrate Addresses from qualifying Orders
WHERE Id IN (
    SELECT Address_A__c FROM Order
    WHERE Billing_Invoice__r.BBF_Ban__c = true
    UNION
    SELECT Address_Z__c FROM Order
    WHERE Billing_Invoice__r.BBF_Ban__c = true
)
```

## Reset for Re-Testing

To reset and re-run:

### Clear BBF_Ban__c in ES UAT
```apex
List<Billing_Invoice__c> bans = [SELECT Id, BBF_Ban__c FROM Billing_Invoice__c WHERE BBF_Ban__c = true];
System.debug('Found ' + bans.size() + ' BANs to reset');
for (Billing_Invoice__c bi : bans) {
    bi.BBF_Ban__c = false;
}
update bans;
```

### Clear BBF_New_Id__c from migrated objects
```apex
// Accounts
List<Account> accounts = [SELECT Id FROM Account WHERE BBF_New_Id__c != null];
for (Account a : accounts) { a.BBF_New_Id__c = null; }
update accounts;

// Contacts
List<Contact> contacts = [SELECT Id FROM Contact WHERE BBF_New_Id__c != null];
for (Contact c : contacts) { c.BBF_New_Id__c = null; }
update contacts;

// Addresses
List<Address__c> addresses = [SELECT Id FROM Address__c WHERE BBF_New_Id__c != null];
for (Address__c a : addresses) { a.BBF_New_Id__c = null; }
update addresses;

// BANs
List<Billing_Invoice__c> bans = [SELECT Id FROM Billing_Invoice__c WHERE BBF_New_Id__c != null];
for (Billing_Invoice__c bi : bans) { bi.BBF_New_Id__c = null; }
update bans;

// Orders
List<Order> orders = [SELECT Id FROM Order WHERE BBF_New_Id__c != null];
for (Order o : orders) { o.BBF_New_Id__c = null; }
update orders;
```